<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_1_mixed_pubmed_cadec_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [ ]:
# training data set

train_csv = path/'AskAPatient.fold-1.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [13]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-1.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heels hurt
2,4,Discoloration of skin [blotchy] on the neck
3,6,diastolic dysfunction
4,8,arthritis


In [14]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-1.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthristis
4,8,arthritis


In [15]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'pubmed_cadec_askpatient_mixed.csv')

In [16]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [17]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [18]:
data_clas.add_test(test,label=0)


In [19]:
data_lm.save()
data_clas.save()

In [ ]:
data_lm.train_ds.vocab.itos

In [21]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [22]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.814389,3.465255,0.437145,33:18


In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.385420,3.237897,0.457963,33:14


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.213119,3.207352,0.461010,33:26


In [26]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.170323,3.187829,0.463238,33:21


In [27]:
learn.save_encoder('pubmed_cadec_askpatient_mixed_fold1_first')

In [28]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_askpatient_mixed_fold1_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.760907,3.670561,0.378935,01:07


In [29]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.693968,2.939465,0.461259,01:19


In [30]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.175892,2.657423,0.521792,03:17


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.824403,2.241655,0.598063,03:22
1,1.549261,2.103423,0.613801,03:28


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.361864,1.953322,0.644068,03:15
1,1.182584,1.839482,0.656174,03:10


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.082295,1.719650,0.679177,03:20
1,0.945938,1.676349,0.688862,03:21


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.860384,1.610853,0.697337,03:27
1,0.787040,1.538715,0.709443,03:15


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.740472,1.579910,0.723971,03:08
1,0.633786,1.496871,0.725182,03:15


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.591511,1.488411,0.732446,03:14
1,0.612314,1.432112,0.746973,03:08
2,0.491620,1.359531,0.757869,03:16
3,0.404030,1.381915,0.760291,03:12


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.473580,1.383710,0.761501,03:27
1,0.456109,1.453178,0.760291,03:10
2,0.386933,1.338599,0.769976,03:17
3,0.343961,1.308345,0.776029,03:20


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.320501,1.377900,0.774818,03:11
1,0.377892,1.306214,0.779661,03:28
2,0.343197,1.260605,0.778450,03:11
3,0.290599,1.297137,0.777240,03:12


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.262502,1.366337,0.773608,03:26
1,0.310046,1.375707,0.773608,03:26
2,0.271579,1.348458,0.777240,03:17
3,0.229172,1.382813,0.778450,03:16


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.216782,1.394087,0.782082,03:09
1,0.262492,1.436045,0.778450,03:23
2,0.218562,1.380545,0.786925,03:05
3,0.203256,1.350662,0.785714,03:17


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.214388,1.360008,0.784504,03:17
1,0.227882,1.507399,0.786925,03:10
2,0.220692,1.444624,0.790557,03:27
3,0.180952,1.393517,0.790557,03:25


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.189623,1.395438,0.795399,03:18
1,0.215227,1.471679,0.789346,03:12
2,0.183408,1.410315,0.800242,03:24
3,0.147890,1.397759,0.796610,03:16


In [43]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.195618,1.381589,0.792978,03:16
1,0.190783,1.410283,0.800242,03:23


In [44]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.162731,1.486659,0.796610,03:23
1,0.168921,1.392320,0.805085,03:26


In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.166198,1.483725,0.799031,03:14


In [46]:
# save the best model

learn.save_encoder('pubmed_cadec_askpatient_mixed_fold1')

# Part three: Predict on the test dataset

In [47]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [48]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[2, 730, 8, 96, 8, 8, 728, 16, 20, 21, 22, 246, 27, 31, 36, 37, 37, 37, 37, 37, 676, 37, 37, 37, 37, 37, 42, 42, 44, 813, 325, 49, 49, 49, 49, 49, 49, 49, 53, 53, 690, 226, 60, 61, 62, 540, 70, 776, 71, 122, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 260, 80, 878, 80, 80, 80, 80, 491, 289, 88, 89, 75, 92, 92, 92, 94, 94, 94, 94, 94, 736, 94, 94, 94, 94, 96, 849, 97, 98, 100, 100, 100, 100, 766, 100, 100, 572, 301, 337, 775, 301, 103, 103, 105, 107, 107, 108, 108, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 857, 117, 142, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 802, 293, 624, 246, 135, 135, 135, 139, 139, 296, 142, 779, 864, 146, 158, 166, 166, 166, 169, 171, 172, 37, 172, 172, 172, 173, 173, 173, 710, 412, 182, 182, 794, 182, 182, 182, 182, 182, 182, 182, 182, 183, 183, 183, 183, 183, 183, 183, 184, 848, 848, 848, 848, 189, 189, 190, 190, 190,

In [49]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

663
0.7647058823529411
